In [1]:
!pip install -q kaggle

In [2]:
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat '/content/kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [3]:
! kaggle datasets download andradaolteanu/gtzan-dataset-music-genre-classification

Dataset URL: https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification
License(s): other
 99% 1.20G/1.21G [00:17<00:00, 103MB/s]
100% 1.21G/1.21G [00:17<00:00, 76.3MB/s]


In [4]:
! unzip /content/gtzan-dataset-music-genre-classification.zip

Archive:  /content/gtzan-dataset-music-genre-classification.zip
  inflating: Data/features_30_sec.csv  
  inflating: Data/features_3_sec.csv  
  inflating: Data/genres_original/blues/blues.00000.wav  
  inflating: Data/genres_original/blues/blues.00001.wav  
  inflating: Data/genres_original/blues/blues.00002.wav  
  inflating: Data/genres_original/blues/blues.00003.wav  
  inflating: Data/genres_original/blues/blues.00004.wav  
  inflating: Data/genres_original/blues/blues.00005.wav  
  inflating: Data/genres_original/blues/blues.00006.wav  
  inflating: Data/genres_original/blues/blues.00007.wav  
  inflating: Data/genres_original/blues/blues.00008.wav  
  inflating: Data/genres_original/blues/blues.00009.wav  
  inflating: Data/genres_original/blues/blues.00010.wav  
  inflating: Data/genres_original/blues/blues.00011.wav  
  inflating: Data/genres_original/blues/blues.00012.wav  
  inflating: Data/genres_original/blues/blues.00013.wav  
  inflating: Data/genres_original/blues/blues

In [66]:
import os
import numpy as np
import librosa
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional


In [27]:


dataset_path = '/content/Data/genres_original'


genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']


In [74]:
n_mfcc = 13
frame_length = 25  # in milliseconds






def extract_features(file_path):
    y, sr = librosa.load(file_path)
    frame_length_samples = int(sr * frame_length / 1000)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc, n_fft=frame_length_samples)
    return np.mean(mfcc, axis=1)

In [75]:


data = []
labels = []

for genre in genres:
    genre_path = os.path.join(dataset_path, genre)
    for file_name in os.listdir(genre_path):
      try:
        file_path = os.path.join(genre_path, file_name)
        features = extract_features(file_path)
        data.append(features)
        labels.append(genres.index(genre))
      except Exception as e:
            print(f"Error loading {file_path}: {str(e)}")




<ipython-input-74-1c9e42c52e35>:10: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error loading /content/Data/genres_original/jazz/jazz.00054.wav: 


In [76]:
data = np.array(data)
labels = np.array(labels)

In [77]:
data.shape

(999, 13)

In [78]:
labels.shape

(999,)

In [79]:
# Encode labels
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)


X_train, X_val, y_train, y_val = train_test_split(data, labels_encoded, test_size=0.2, random_state=42)


In [80]:
X_train.shape

(799, 13)

In [81]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)

In [82]:
X_train.shape

(799, 13, 1)

In [83]:
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(n_mfcc, 1)),
    LSTM(64),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='softmax')
])

In [84]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_21 (LSTM)              (None, 13, 64)            16896     
                                                                 
 lstm_22 (LSTM)              (None, 64)                33024     
                                                                 
 dense_22 (Dense)            (None, 64)                4160      
                                                                 
 dropout_11 (Dropout)        (None, 64)                0         
                                                                 
 dense_23 (Dense)            (None, 10)                650       
                                                                 
Total params: 54730 (213.79 KB)
Trainable params: 54730 (213.79 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [95]:

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val))


train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']



Epoch 1/30
25/25 [==============================] - 11s 106ms/step - loss: 1.7345 - accuracy: 0.3930 - val_loss: 1.7598 - val_accuracy: 0.3450
Epoch 2/30
25/25 [==============================] - 1s 32ms/step - loss: 1.7157 - accuracy: 0.3767 - val_loss: 1.7519 - val_accuracy: 0.3200
Epoch 3/30
25/25 [==============================] - 1s 34ms/step - loss: 1.7037 - accuracy: 0.4018 - val_loss: 1.7507 - val_accuracy: 0.3300
Epoch 4/30
25/25 [==============================] - 1s 43ms/step - loss: 1.6911 - accuracy: 0.4005 - val_loss: 1.7343 - val_accuracy: 0.3400
Epoch 5/30
25/25 [==============================] - 2s 61ms/step - loss: 1.6829 - accuracy: 0.3917 - val_loss: 1.7283 - val_accuracy: 0.3300
Epoch 6/30
25/25 [==============================] - 2s 61ms/step - loss: 1.6682 - accuracy: 0.4055 - val_loss: 1.7236 - val_accuracy: 0.3400
Epoch 7/30
25/25 [==============================] - 1s 33ms/step - loss: 1.6892 - accuracy: 0.4030 - val_loss: 1.7086 - val_accuracy: 0.3550
Epoch 8/30


In [96]:

for epoch in range(30):
    print(f'Epoch {epoch + 1}: Train Accuracy = {train_accuracy[epoch]}, Validation Accuracy = {val_accuracy[epoch]}')


Epoch 1: Train Accuracy = 0.39299124479293823, Validation Accuracy = 0.3449999988079071
Epoch 2: Train Accuracy = 0.3767209053039551, Validation Accuracy = 0.3199999928474426
Epoch 3: Train Accuracy = 0.40175220370292664, Validation Accuracy = 0.33000001311302185
Epoch 4: Train Accuracy = 0.400500625371933, Validation Accuracy = 0.3400000035762787
Epoch 5: Train Accuracy = 0.3917396664619446, Validation Accuracy = 0.33000001311302185
Epoch 6: Train Accuracy = 0.4055068790912628, Validation Accuracy = 0.3400000035762787
Epoch 7: Train Accuracy = 0.4030037522315979, Validation Accuracy = 0.35499998927116394
Epoch 8: Train Accuracy = 0.40675845742225647, Validation Accuracy = 0.3499999940395355
Epoch 9: Train Accuracy = 0.41051313281059265, Validation Accuracy = 0.3799999952316284
Epoch 10: Train Accuracy = 0.42803505063056946, Validation Accuracy = 0.35499998927116394
Epoch 11: Train Accuracy = 0.41051313281059265, Validation Accuracy = 0.35499998927116394
Epoch 12: Train Accuracy = 0.42

# part 2

In [97]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_mfcc, 1)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(len(genres), activation='softmax'))




In [98]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_26 (LSTM)              (None, 128)               66560     
                                                                 
 dense_28 (Dense)            (None, 64)                8256      
                                                                 
 dropout_14 (Dropout)        (None, 64)                0         
                                                                 
 dense_29 (Dense)            (None, 10)                650       
                                                                 
Total params: 75466 (294.79 KB)
Trainable params: 75466 (294.79 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [99]:

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val))


train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

Epoch 1/30
25/25 [==============================] - 6s 83ms/step - loss: 2.2502 - accuracy: 0.1402 - val_loss: 2.2190 - val_accuracy: 0.1750
Epoch 2/30
25/25 [==============================] - 1s 21ms/step - loss: 2.1986 - accuracy: 0.2015 - val_loss: 2.1741 - val_accuracy: 0.1950
Epoch 3/30
25/25 [==============================] - 1s 23ms/step - loss: 2.1601 - accuracy: 0.2165 - val_loss: 2.1419 - val_accuracy: 0.2800
Epoch 4/30
25/25 [==============================] - 0s 18ms/step - loss: 2.1171 - accuracy: 0.2778 - val_loss: 2.1154 - val_accuracy: 0.2450
Epoch 5/30
25/25 [==============================] - 0s 19ms/step - loss: 2.0950 - accuracy: 0.2804 - val_loss: 2.0885 - val_accuracy: 0.2650
Epoch 6/30
25/25 [==============================] - 0s 18ms/step - loss: 2.0689 - accuracy: 0.2791 - val_loss: 2.0639 - val_accuracy: 0.2750
Epoch 7/30
25/25 [==============================] - 1s 20ms/step - loss: 2.0402 - accuracy: 0.2854 - val_loss: 2.0426 - val_accuracy: 0.2850
Epoch 8/30
25

In [100]:

for epoch in range(30):
    print(f'Epoch {epoch + 1}: Train Accuracy = {train_accuracy[epoch]}, Validation Accuracy = {val_accuracy[epoch]}')


Epoch 1: Train Accuracy = 0.1401752233505249, Validation Accuracy = 0.17499999701976776
Epoch 2: Train Accuracy = 0.20150187611579895, Validation Accuracy = 0.19499999284744263
Epoch 3: Train Accuracy = 0.21652065217494965, Validation Accuracy = 0.2800000011920929
Epoch 4: Train Accuracy = 0.2778473198413849, Validation Accuracy = 0.24500000476837158
Epoch 5: Train Accuracy = 0.2803504467010498, Validation Accuracy = 0.26499998569488525
Epoch 6: Train Accuracy = 0.27909886837005615, Validation Accuracy = 0.2750000059604645
Epoch 7: Train Accuracy = 0.28535670042037964, Validation Accuracy = 0.2849999964237213
Epoch 8: Train Accuracy = 0.3078848421573639, Validation Accuracy = 0.2750000059604645
Epoch 9: Train Accuracy = 0.3128911256790161, Validation Accuracy = 0.29499998688697815
Epoch 10: Train Accuracy = 0.2978723347187042, Validation Accuracy = 0.2849999964237213
Epoch 11: Train Accuracy = 0.32665830850601196, Validation Accuracy = 0.28999999165534973
Epoch 12: Train Accuracy = 0.3

In [101]:
model = Sequential()
model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(n_mfcc, 1)))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(len(genres), activation='softmax'))

In [102]:
model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_4 (Bidirecti  (None, 13, 128)           33792     
 onal)                                                           
                                                                 
 bidirectional_5 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 dense_30 (Dense)            (None, 64)                8256      
                                                                 
 dropout_15 (Dropout)        (None, 64)                0         
                                                                 
 dense_31 (Dense)            (None, 10)                650       
                                                                 
Total params: 141514 (552.79 KB)
Trainable params: 14

In [105]:

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val))


train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

Epoch 1/30
25/25 [==============================] - 11s 105ms/step - loss: 1.7917 - accuracy: 0.3630 - val_loss: 1.8385 - val_accuracy: 0.3050
Epoch 2/30
25/25 [==============================] - 1s 33ms/step - loss: 1.7732 - accuracy: 0.3655 - val_loss: 1.8269 - val_accuracy: 0.3350
Epoch 3/30
25/25 [==============================] - 1s 32ms/step - loss: 1.7680 - accuracy: 0.3830 - val_loss: 1.8211 - val_accuracy: 0.3450
Epoch 4/30
25/25 [==============================] - 1s 31ms/step - loss: 1.7569 - accuracy: 0.3592 - val_loss: 1.8258 - val_accuracy: 0.3100
Epoch 5/30
25/25 [==============================] - 1s 33ms/step - loss: 1.7483 - accuracy: 0.3780 - val_loss: 1.8210 - val_accuracy: 0.3350
Epoch 6/30
25/25 [==============================] - 1s 32ms/step - loss: 1.7319 - accuracy: 0.3767 - val_loss: 1.8183 - val_accuracy: 0.3150
Epoch 7/30
25/25 [==============================] - 1s 50ms/step - loss: 1.7125 - accuracy: 0.3905 - val_loss: 1.8031 - val_accuracy: 0.3450
Epoch 8/30


In [106]:

for epoch in range(30):
    print(f'Epoch {epoch + 1}: Train Accuracy = {train_accuracy[epoch]}, Validation Accuracy = {val_accuracy[epoch]}')


Epoch 1: Train Accuracy = 0.36295369267463684, Validation Accuracy = 0.3050000071525574
Epoch 2: Train Accuracy = 0.36545681953430176, Validation Accuracy = 0.33500000834465027
Epoch 3: Train Accuracy = 0.38297873735427856, Validation Accuracy = 0.3449999988079071
Epoch 4: Train Accuracy = 0.35919898748397827, Validation Accuracy = 0.3100000023841858
Epoch 5: Train Accuracy = 0.37797245383262634, Validation Accuracy = 0.33500000834465027
Epoch 6: Train Accuracy = 0.3767209053039551, Validation Accuracy = 0.3149999976158142
Epoch 7: Train Accuracy = 0.3904881179332733, Validation Accuracy = 0.3449999988079071
Epoch 8: Train Accuracy = 0.3679599463939667, Validation Accuracy = 0.3400000035762787
Epoch 9: Train Accuracy = 0.3917396664619446, Validation Accuracy = 0.33500000834465027
Epoch 10: Train Accuracy = 0.39299124479293823, Validation Accuracy = 0.32499998807907104
Epoch 11: Train Accuracy = 0.38923653960227966, Validation Accuracy = 0.3449999988079071
Epoch 12: Train Accuracy = 0.3